In [4]:
# import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta


In [5]:
df = pd.read_csv('BA.csv')

df['Date'] = pd.to_datetime(df['Date'])

df.index = df['Date']

df.head()
# df.describe()


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-02-03,2015-02-03,146.369995,148.000000,146.179993,147.339996,128.639938,5978200
2015-02-04,2015-02-04,145.970001,148.389999,145.929993,147.220001,128.535172,4766200
2015-02-05,2015-02-05,147.259995,148.869995,146.460007,148.600006,129.740036,4252600
2015-02-06,2015-02-06,148.619995,148.889999,147.699997,148.000000,129.216217,4398800
2015-02-09,2015-02-09,146.160004,148.160004,145.630005,147.580002,128.849487,4202900


In [6]:
df = df.drop(["Date"], axis=1)

In [7]:
#5.1. Step 1: extract technical indicators

#Define Technical Indicators
#_______________________________________________________________________

#Simple Moving Average  
sma = ta.SMA(df.Close,900)
df['MA25'] = sma
df = df.fillna(0)

# RSI
df["RSI"] = ta.RSI(df.Close,14)

# Stochastic K
df["Stoch_K"], df["Stoch_D"] = ta.STOCH(df["High"], df["Low"], df["Close"], fastk_period=14, slowk_period=3, slowk_matype=0, 
                                       slowd_period=3, slowd_matype=0)

# MACD
df['MACD'], df['MACDSignal'], df['MACDHist'] = ta.MACD(df.Close, fastperiod= 12, slowperiod= 26, signalperiod= 9)

# Williams R%
df["WILLR"] = ta.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14) 

#df["WILLR"] = (WILLR - WILLR.min())/(WILLR.max() - WILLR.min())
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR
Date,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,284.427944,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,284.638578,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,284.852078,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,285.067200,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,285.279755,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512


In [8]:
# 5.2. Step 2: trend analysis using technical indicators

#make MA25 into a series
MA25 = df.iloc[:,6]

# Up trend is 1, No trend is 0, and Down trend is -1
df['Trend'] = MA25.rolling('5d').apply(lambda x: np.sign(x[-1] - x[0]))

df.tail()
#If close > MA25 and MA25 is rising for last 5 days then 1 (Uptrend)
#If close < MA25 and MA25 is declining for last 5 days then -1 (Downtrend)
#If neither are true then 0 (Notrend)
def f(row):
    if row['Close'] > row['MA25'] and row['Trend'] == 1:
        val = 1
    elif row['Close'] < row['MA25'] and row['Trend'] == -1:
        val = -1
    else:
        val = 0
    return val
df['Trend'] = df.apply(f, axis=1)
# df.tail(n=1)

C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


In [11]:
#5.3. Step 3: trading signal generation from trend analysis

Close = df.iloc[:,3]
df["UpTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5 + .5
df["DownTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5

def g(row):
    if row['Trend'] == 1:
        val = row['UpTrend']
    elif row['Trend'] == -1:
        val = row['DownTrend']
    else:
        val = 0
    return val
df['Trade_Signal'] = df.apply(g, axis=1)
df

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend,UpTrend,DownTrend,Trade_Signal
Date,,,,,,,,,,,,,,,,,,
2015-02-03,146.369995,148.000000,146.179993,147.339996,128.639938,5978200,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0.000000
2015-02-04,145.970001,148.389999,145.929993,147.220001,128.535172,4766200,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.500000,0.000000,0.000000
2015-02-05,147.259995,148.869995,146.460007,148.600006,129.740036,4252600,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.000000,0.500000,0.000000
2015-02-06,148.619995,148.889999,147.699997,148.000000,129.216217,4398800,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.782607,0.282607,0.000000
2015-02-09,146.160004,148.160004,145.630005,147.580002,128.849487,4202900,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0.000000
2015-02-10,148.199997,148.710007,147.500000,148.039993,129.251099,4034900,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.000000,0.500000,0.000000
2015-02-11,147.509995,148.380005,147.089996,147.919998,129.945084,3272700,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.869568,0.369568,0.000000
2015-02-12,148.259995,148.570007,147.679993,148.089996,130.094467,2998900,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.000000,0.500000,0.000000
2015-02-13,147.979996,149.839996,147.639999,149.729996,131.535156,3418600,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.000000,0.500000,0.000000


In [10]:
#I added .tail(n=Length - 32) to omit values where the indicators = 0 since the machine learning agorithm trains on the values 
#of the indicators and the MACD doesn't generate values until 33 periods have passed.
Length = (df.shape[0] - 1100)


df2 = (df[['MA25', 'RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trade_Signal']].copy().tail(n=Length).copy())


#I get some na values without this and I'm worried that using this and
# having those values as zero will mess with the algorithms interpretation of the data
df2 = df2.fillna(0)

# df2.describe()
df2.head()

,MA25,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
Date,,,,,,,
2019-06-18,246.623167,64.906244,82.218373,69.342184,-0.811875,-1.568902,1.000000
2019-06-19,246.867922,59.872392,88.992382,80.116578,0.427263,-22.101158,0.858342
2019-06-20,247.118633,63.445633,89.074074,86.761609,1.897389,-9.107719,1.000000
2019-06-21,247.365789,60.648081,84.278800,87.448419,2.785066,-15.954722,0.759493
2019-06-24,247.616511,61.926708,87.858293,87.070389,3.620311,-11.362681,0.000000


In [ ]:
#5.5. Step 5: network structure creation and training using ELM

target_column = ['Trade_Signal'] 
predictors = list(set(list(df2.columns))-set(target_column))
#line 3 here fufills the data normalization of Step 5.4
df2[predictors] = (df2[predictors] - df2[predictors].min())/(df2[predictors].max() - df2[predictors].min())

#df2.describe()
#df2.info()
df2.tail()
# df2.head()

In [ ]:
X = df2[predictors]

y = df2[target_column]

y = y.T

print(X.shape); print(X.ndim); print(y.shape); print(y.ndim)
# print(X)

In [ ]:
Xindex = np.array([[1], [2], [3], [4], [5], [6]])

Xindex = Xindex.T

print(Xindex.shape); print(Xindex)

In [ ]:
#  Functional Expansion Block

# import sys
# np.set_printoptions(threshold=sys.maxsize)

feb = np.outer(X, Xindex)
# .flatten(order='F')
print(feb.shape); print(feb);
# print(feb.type)

In [ ]:
feb = feb.reshape(-1, 6)
print(feb)

In [ ]:
feb =np.sum(feb, axis=1)
print(feb.shape); print(feb); 

In [ ]:
# feb = np.tanh(feb)
# print(feb)

In [ ]:
#  Functional Expansion Block

# feb = np.outer(X, Xindex).flatten(order='F')
# feb = np.sum(cx.reshape(-1, 6), axis=1)
# feb = np.tanh(feb)

#all on one line
feb = np.tanh(np.sum(np.outer(X, Xindex).flatten(order='F').reshape(-1, 6), axis=1))

print(feb.shape); print(feb.ndim);
# print(feb)

In [ ]:
cx = feb.reshape(1, len(feb) )
print(cx.shape); print(cx.ndim);
# print(cx)

In [ ]:
# initialize random weights
Y = np.random.random((1, (Length * 6)))

print(Y.shape); print(Y.ndim);print(Y)

In [ ]:
I = np.identity(len(feb))

print(I.shape); print(I.ndim); print(I)

In [ ]:
# W= (cx.T).dot(cx)

# lamb = 1
# W = np.linalg.lstsq(cx.T.dot(cx) + lamb * I, cx.T.dot(Y),  rcond=1)

# W = ((cx.T.dot(cx) + lamb * I)**-1) * (cx.T.dot(Y))



lamb = 1
W = np.linalg.lstsq(cx.T.dot(cx) + lamb * I, cx.T.dot(Y))[0]  

# 
print(W.shape);  print(W.ndim); print(W)


In [ ]:
cx_index = np.array(len(feb))
print(cx.shape);  print(cx.ndim);
# print(cx)

In [ ]:

y = np.outer(W, cx_index).reshape(948, -1)
# print(y)

In [ ]:
# y= np.outer(W, cx_index).flatten(order='F')
# print(y.shape)

In [ ]:
y = np.sum(np.outer(W, cx).reshape(948, -1), axis=1)
# print(y.shape)

In [ ]:
print(y.shape)

In [ ]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)

print(y)

In [ ]:
Outputnp.mean(a, axis=1)

In [ ]:
#I'm working on figure 1 of the article and I have the expanded inputs CX, and the initialized weights as z
#I don't know how to read equation 2 or the Moore-Penros generalized inverse function of equation 5. If I'm reading this correctly
#(I'm not 100% sure I am), those functions are the key to using my CX expanded output values to create W.

# I'm still new to ELM so if I'm missing something please let me know. 
#any help is appreciated
#Thank you so much !!!!

In [ ]:
#this is a for loop  I found online. It is not what is described in the article

for iteration in range(1000):
    z = np.dot(cx.T, Y)
    sigmoid = 1/(1+np.exp(-cx))
    error = (y - sigmoid)
print(error); print(error.shape)

In [ ]:
sigmoidDerivative = sigmoid * (1 - sigmoid)
print(sigmoidDerivative)

In [ ]:
synaptic_weights += np.dot(cx, error*sigmoidDerivative)
print(synaptic_weights)